For this workshop, you need:

* An Azure Machine Learning workspace. 
* The Azure Machine Learning Python SDK v2 installed. 

To install the SDK you can either,

Create a compute instance, which already has installed the latest AzureML Python SDK and is pre-configured for ML workflows.

Use the followings commands to install Azure ML Python SDK v2:

```bash
pip install azure-ai-ml --upgrade
```

## Connect to ML Client

To connect to a workspace, you need to provide a subscription, resource group and workspace name. These details are used in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In the following example, the default Azure authentication is used along with the default workspace configuration or from any `config.json` file you might have copied into the folders structure. If no `config.json` is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating `MLClient`.

```python
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
```


In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential, path="config.json")

# Model Training

## 1. Create Managed Compute

A compute is a designated compute resource where you run your job or host your endpoint. Azure Machine learning supports the following types of compute:

- **Compute instance** - a fully configured and managed development environment in the cloud. You can use the instance as a training or inference compute for development and testing. It's similar to a virtual machine on the cloud.

- **Compute cluster** - a managed-compute infrastructure that allows you to easily create a cluster of CPU or GPU compute nodes in the cloud.

- **Inference cluster** - used to deploy trained machine learning models to Azure Kubernetes Service. You can create an Azure Kubernetes Service (AKS) cluster from your Azure ML workspace, or attach an existing AKS cluster.

- **Attached compute** - You can attach your own compute resources to your workspace and use them for training and inference.

You can create a compute using the Studio, the cli and the sdk.

<hr>

We can create a **compute instance** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_compute_instance.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


<hr>

We can create a **compute cluster** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_compute_cluster.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>


Let's create a managed compute cluster for the training workload.

In [ ]:
# Retrieve an already attached Azure Machine Learning Compute.
cpu_compute_target = "cpu-cluster"
print(ml_client.compute.get(cpu_compute_target))
gpu_compute_target = "gpu-cluster"
print(ml_client.compute.get(gpu_compute_target))

In [ ]:
from azure.ai.ml.entities import AmlCompute

my_cluster = AmlCompute(
    name="cpu-cluster",
    type="amlcompute", 
    size="STANDARD_DS3_V2", 
    min_instances=0, 
    max_instances=4,
    location="westeurope", 	
)

ml_client.compute.begin_create_or_update(my_cluster)


## 2. Register Data Asset

**Datastore** - Azure Machine Learning Datastores securely keep the connection information to your data storage on Azure, so you don't have to code it in your scripts.

An Azure Machine Learning datastore is a **reference** to an **existing** storage account on Azure. The benefits of creating and using a datastore are:
A common and easy-to-use API to interact with different storage type. 
Easier to discover useful datastores when working as a team.
When using credential-based access (service principal/SAS/key), the connection information is secured so you don't have to code it in your scripts.

Supported Data Resources: 

* Azure Storage blob container
* Azure Storage file share
* Azure Data Lake Gen 1
* Azure Data Lake Gen 2
* Azure SQL Database 
* Azure PostgreSQL Database
* Azure MySQL Database

It is not a requirement to use Azure Machine Learning datastores - you can use storage URIs directly assuming you have access to the underlying data.

You can create a datastore using the Studio, the cli and the sdk.

<hr>

We can create a **datastore** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_datastore.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>



**Data asset** - Create data assets in your workspace to share with team members, version, and track data lineage.

By creating a data asset, you create a reference to the data source location, along with a copy of its metadata. 

The benefits of creating data assets are:

* You can **share and reuse data** with other members of the team such that they do not need to remember file locations.
* You can **seamlessly access data** during model training (on any supported compute type) without worrying about connection strings or data paths.
* You can **version** the data.

<hr>

We can create a **data asset** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_data_asset.png" width = "700px" alt="Create Compute Instance cli vs sdk">
</center>

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="../../data/taxi-data.csv",
    type=AssetTypes.URI_FILE,
    description="Taxi dataset",
    name="taxi-data"
)
ml_client.data.create_or_update(my_data)

## 3. Register Train Environment

Azure Machine Learning environments define the execution environments for your **jobs** or **deployments** and encapsulate the dependencies for your code. 

Azure ML uses the environment specification to create the Docker container that your **training** or **scoring code** runs in on the specified compute target.

Create an environment from a
* conda specification
* Docker image
* Docker build context

There are two types of environments in Azure ML: **curated** and **custom environments**. Curated environments are predefined environments containing popular ML frameworks and tooling. Custom environments are user-defined.

<hr>

We can register an **environment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_environment.png" width = "700px" alt="Create Compute Cluster cli vs sdk">
</center>

In [ ]:
from azure.ai.ml.entities import Environment

my_environment = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="../../data-science/environment/train-conda.yml",
    name="taxi-train-env",
    description="Environment created from a Docker image plus Conda environment to train taxi model.",
)

ml_client.environments.create_or_update(my_environment)

## 4. Create Pipeline Job

**AML Job**:

Azure ML provides several ways to train your models, from code-first solutions to low-code solutions:

* Azure ML supports script files in python, R, Java, Julia or C#. All you need to learn is YAML format and command lines to use Azure ML.

* Distributed Training: AML supports integrations with popular frameworks, PyTorch and TensorFlow. Both frameworks employ data parallelism & model parallelism for distributed training.

* Automated ML - Train models without extensive data science or programming knowledge

* Designer - drag and drop web-based UI

<hr>

We can submit a **job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_job.png" width = "700px" alt="Create Compute Cluster cli vs sdk">
</center>

<br>
    
**AML Pipelines**:

An AML pipeline is an independently executable workflow of a complete machine learning task. It helps standardizing the best practices of producing a machine learning model: The core of a machine learning pipeline is to split a complete machine learning task into a multistep workflow. Each step is a manageable component that can be developed, optimized, configured, and automated individually. 

<hr>

We can submit a **pipeline job** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../imgs/create_pipeline.png" width = "700px" alt="Create Compute Cluster cli vs sdk">
</center>

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input, Output, load_component
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# Create pipeline job
parent_dir = "../../data-science"

# 1. Load components
prepare_data = load_component(source=parent_dir + "/prep.yml")
train_model = load_component(source=parent_dir + "/train.yml")
evaluate_model = load_component(source=parent_dir + "/evaluate.yml")
register_model = load_component(source=parent_dir + "/register.yml")

# 2. Construct pipeline
@pipeline()
def taxi_training_pipeline(raw_data, enable_monitoring, table_name):
    
    prepare = prepare_data(
        raw_data=raw_data,
        enable_monitoring=enable_monitoring, 
        table_name=table_name
    )

    train = train_model(
        train_data=prepare.outputs.train_data
    )

    evaluate = evaluate_model(
        model_name="taxi-model",
        model_input=train.outputs.model_output,
        test_data=prepare.outputs.test_data
    )


    register = register_model(
        model_name="taxi-model",
        model_path=train.outputs.model_output,
        evaluation_output=evaluate.outputs.evaluation_output
    )

    return {
        "pipeline_job_train_data": prepare.outputs.train_data,
        "pipeline_job_test_data": prepare.outputs.test_data,
        "pipeline_job_trained_model": train.outputs.model_output,
        "pipeline_job_score_report": evaluate.outputs.evaluation_output,
    }


pipeline_job = taxi_training_pipeline(
    Input(type=AssetTypes.URI_FILE, path="taxi-data@latest"), "false", "taximonitoring"
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

# Model Online Endpoint

## 1. Create Online Endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
online_endpoint = ManagedOnlineEndpoint(
    name="taxi-online-endpoint-2", 
    description="Taxi online endpoint",
    auth_mode="key",
)
ml_client.online_endpoints.begin_create_or_update(
    online_endpoint,   
)


## 2. Create Online Deployment

In [ ]:
# create online deployment
from azure.ai.ml.entities import ManagedOnlineDeployment, Model, Environment

model = "taxi-model@latest"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name="taxi-online-endpoint-2",
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment
)


## 3. Allocate Traffic

In [ ]:
# allocate traffic
# blue deployment takes 100 traffic
online_endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(online_endpoint)

## 4. Invoke and Test Endpoint

In [ ]:
# invoke and test endpoint
ml_client.online_endpoints.invoke(
    endpoint_name="taxi-online-endpoint-2",
    request_file="../../data/taxi-request.json",
)


# Model Batch Endpoint

## 1. Create Batch Compute Cluster

In [ ]:
# create compute cluster to be used by batch cluster
from azure.ai.ml.entities import AmlCompute

my_cluster = AmlCompute(
    name="batch-cluster",
    type="amlcompute", 
    size="STANDARD_DS3_V2", 
    min_instances=0, 
    max_instances=3,
    location="westeurope", 	
)
ml_client.compute.begin_create_or_update(my_cluster)

## 2. Create Batch Endpoint

In [ ]:
# create batch endpoint
from azure.ai.ml.entities import BatchEndpoint

batch_endpoint = BatchEndpoint(
    name="taxi-batch-endpoint-2",
    description="Taxi batch endpoint",
    tags={"model": "taxi-model@latest"},
)

ml_client.begin_create_or_update(batch_endpoint)


## 3. Create Batch Deployment

In [ ]:
# create batch deployment
from azure.ai.ml.entities import BatchDeployment, Model, Environment
from azure.ai.ml.constants import BatchDeploymentOutputAction

model = "taxi-model@latest"

batch_deployment = BatchDeployment(
    name="taxi-batch-dp",
    description="this is a sample batch deployment",
    endpoint_name="taxi-batch-endpoint-2",
    model=model,
    compute="batch-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
)

ml_client.begin_create_or_update(batch_deployment)


## 5. Set deployment as the default deployment in the endpoint:

In [ ]:
batch_endpoint = ml_client.batch_endpoints.get("taxi-batch-endpoint-2")
batch_endpoint.defaults.deployment_name = batch_deployment.name
ml_client.batch_endpoints.begin_create_or_update(batch_endpoint)

## 4. Invoke and Test Endpoint

In [ ]:
# invoke and test endpoint
from azure.ai.ml import Input

input = Input(path="../../data/taxi-batch.csv", 
              type=AssetTypes.URI_FILE, 
              mode=InputOutputModes.DOWNLOAD)


# invoke the endpoint for batch scoring job
ml_client.batch_endpoints.invoke(
    endpoint_name="taxi-batch-endpoint",
    input=input,
    deployment_name="taxi-batch-dp"
)
